<h1>8 Data Wrangling: Join, Combine, and Reshape</h1>

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
pd.options.display.max_columns = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

<h2>8.1 Hierachical Indexing</h2>
<em>Hierarchical indexing</em> is an important feature of pandas that enables you to have multiple (two or more) index levels on an axis. Another way of thinking about it is that it provides a way for you to work with higher dimensional data in a lower dimensional form.

In [2]:
# Create a Series with a list of lists (or arrays) as the index
data = pd.Series(np.random.uniform(size=9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.929616
   2    0.316376
   3    0.183919
b  1    0.204560
   3    0.567725
c  1    0.595545
   2    0.964515
d  2    0.653177
   3    0.748907
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

With a hierarchically indexed object, so-called partial indexing is possible, enabling you to concisely select subsets of the data:

In [4]:
data["b"]

1    0.204560
3    0.567725
dtype: float64

In [5]:
data["b":"c"]

b  1    0.204560
   3    0.567725
c  1    0.595545
   2    0.964515
dtype: float64

In [6]:
data.loc[["b", "d"]]

b  1    0.204560
   3    0.567725
d  2    0.653177
   3    0.748907
dtype: float64

Selection is even possible from an 'inner' level.

In [7]:
data.loc[:, 2]

a    0.316376
c    0.964515
d    0.653177
dtype: float64

Hierarchical indexing plays an important role in reshaping data and in group-based operations like forming a pivot table. For example, you can rearrange this data into a DataFrame using its <code>unstack</code> method:

In [8]:
data.unstack()

,1,2,3
a,0.929616,0.316376,0.183919
b,0.204560,NaN,0.567725
c,0.595545,0.964515,NaN
d,NaN,0.653177,0.748907


The inverse operation of <code>unstack</code> is <code>stack</code>:

In [9]:
data.unstack().stack()

a  1    0.929616
   2    0.316376
   3    0.183919
b  1    0.204560
   3    0.567725
c  1    0.595545
   2    0.964515
d  2    0.653177
   3    0.748907
dtype: float64

With a DataFrame, either axis can have a hierarchical index:

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"],
                              ["Green", "Red", "Green"]])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

The hierarchical levels can have names

In [11]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

<p>You can see how many levels an index has by accessing its <code>nlevels</code> attribute:</p>

In [12]:
frame.index.nlevels, frame.columns.nlevels

(2, 2)

Select groups of columns:

In [13]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [14]:
frame.iloc[:, frame.columns.get_level_values(1)=='Green']

state      Ohio Colorado
color     Green    Green
key1 key2               
a    1        0        2
     2        3        5
b    1        6        8
     2        9       11

A <code>MultiIndex</code> can be created by itself and then reused; the columns in the preceding DataFrame with level names could also be created like this:

In [15]:
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "Colorado"],
                          ["Green", "Red", "Green"]],
                          names=["state", "color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

<h3>Reordering and Sorting Levels</h3>
<p>At times you may need to rearrange the order of the levels on an axis or sort the data by the values in one specific level. The <code>swaplevel</code> method takes two level numbers or names and returns a new object with the levels interchanged (but the data is otherwise unaltered):</p>

In [16]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

<p>At times you may need to rearrange the order of the levels on an axis or sort the data by the values in one specific level. The <code>swaplevel</code> method takes two level numbers or names and returns a new object with the levels interchanged (but the data is otherwise unaltered):</p>

In [17]:
frame.sort_index(level=1) # sorting by 2nd index or key2

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [18]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

<h3>Summary Statistics by Level</h3>
<p>Many descriptive and summary statistics on DataFrame and Series have a <code>level</code> option in which you can specify the level you want to aggregate by on a particular axis. Consider the above DataFrame; we can aggregate by level on either the rows or columns, like so:</p>

In [19]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [20]:
frame.groupby(level="color", axis="columns").sum()

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

<h3>Indexing with a DataFrame's columns</h3>
<p>It’s not unusual to want to use one or more columns from a DataFrame as the row index; alternatively, you may wish to move the row index into the DataFrame’s columns. Here’s an example DataFrame:</p>

In [21]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                      "c": ["one", "one", "one", "two", "two",
                            "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame’s <code>set_index</code> function will create a new DataFrame using one or more of its columns as the index:

In [22]:
frame2 = frame.set_index(["c", "d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

By default, the columns are removed from the DataFrame, though you can leave them in by passing <code>drop=False</code> to <code>set_index</code>:

In [23]:
frame.set_index(["c", "d"], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

<code>reset_index</code>, on the other hand, does the opposite of <code>set_index</code>; the hierarchical index levels are moved into the columns:

In [24]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


<h2>8.2 Combining and Merging Datasets

<p>Data contained in pandas objects can be combined in a number of ways:</p>

<dl>
    <dt><code>pandas.merge</code></dt>
    <dd>Connect rows in DataFrames based on one or more keys. This will be familiar to users of SQL or other relational databases, as it implements database join operations.</dd>
    <dt><code>pandas.concat</code></dt>
    <dd>Concatenate or "stack" objects together along an axis.</dd>
    <dt><code>combine_first</code></dt>
    <dd>Splice together overlapping data to fill in missing values in one object with values from another.</dd>
</dl>

<p>I will address each of these and give a number of examples. They’ll be utilized in examples throughout the rest of the book.</p>

<h3>Database-Style DataFrame Joins</h3>

<p><em>Merge</em> or <em>join</em> operations combine datasets by linking rows using one or more keys. These operations are particularly important in relational databases (e.g., SQL-based). The <code>pandas.merge</code> function in pandas is the main entry point for using these algorithms on your data.</p>

</p>Let’s start with a simple example:</p>

In [25]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [26]:
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})
df2

,key,data2
0,a,0
1,b,1
2,d,2


<p>Here I am using pandas's Int64 extension type for nullable integers</p>

<p>This is an example of a <em>many-to-one</em> join; the data in <code>df1</code> has multiple rows labeled <code>a</code> and <code>b</code>, whereas <code>df2</code> has only one row for each value in the key column. Calling <code>pandas.merge</code> with these objects, we obtain:

In [27]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


<p>Note that I didn’t specify which column to join on. If that information is not specified, <code>pandas.merge</code> uses the overlapping column names as the keys. It’s a good practice to specify explicitly, though:</p>

In [28]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


<p>If the column names are different in each object, you can specify them separately:</p>

In [29]:
df3 = pd.DataFrame({"lkey": 
                    ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": 
                    pd.Series(range(7), dtype="Int64")})
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


<p>You may notice that the <code>c</code> and <code>d</code> values and associated data are missing from the result. By default, <code>pandas.merge</code> does an <code>inner</code> join; the keys in the result are the intersection, or the common set found in both tables. Other possible options are <code>left</code>, <code>right</code>, and <code>outer</code>. The outer join takes the <strong>union</strong> of the keys, combining the effect of applying both left and right joins:</p>

In [30]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0
6,c,3,<NA>
7,d,<NA>,2


In [31]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


<p>In an outer join, rows from the left or right DataFrame objects that do not match on keys in the other DataFrame will appear with NA values in the other DataFrame's columns for the nonmatching rows.</p>

<p>Summary of the options for <code>how</code>.</p>
<table id='Table_8.1'>
    <tr>
        <th>Option</th>
        <th>Behavior</th>
    </tr>
    <tr>
        <td><code>how="inner"</code></td>
        <td>Use only the key combinations observed in both tables</td>
    </tr>
    <tr>
        <td><code>how="left"</code></td>
        <td>Use all key combinations found in the left table</td>
    </tr>    
    <tr>
        <td><code>how="right"</code></td>
        <td>Use all key combinations found in the right table</td>
    </tr>    
    <tr>
        <td><code>how="outer"</code></td>
        <td>Use all key combinations observed in both tables together</td>
    </tr>
    <caption>Table 8.1: Different join types with the <code>how</code> argument</caption>
</table>

<em>Many-to-many</em> merges form the Cartesian product of the matching keys. Here’s an example:

In [32]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                    "data1": pd.Series(range(6), dtype="Int64")})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [33]:
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
                    "data2": pd.Series(range(5), dtype="Int64")})
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [34]:
pd.merge(df1, df2, on="key", how="left")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


<p>Since there were three <code>b</code> rows in the left DataFrame and two in the right one, there are six <code>b</code> rows in the result, two <code>a</code> rows in <code>df1</code> and two in <code>df2</code>, there are four <code>a</code> in the result. The join method passed to the how keyword argument affects only the distinct key values appearing in the result:

In [35]:
pd.merge(df1, df2, how="inner")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


<p>To merge with multiple keys, pass a list of column names:</p>

In [36]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2": ["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3], dtype='Int64')})
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [37]:
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [38]:
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,<NA>
3,bar,one,3,6
4,bar,two,<NA>,7


<p>To determine which key combinations will appear in the result depending on the choice of merge method, think of the multiple keys as forming an array of tuples to be used as a single join key.</p>

<p>A last issue to consider in merge operations is the treatment of overlapping column names. For example:</p>

In [39]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


<p>While you can address the overlap manually (Ch 7.2.4: Renaming Axis Indexes for renaming axis labels), <code>pandas.merge</code> has a <code>suffixes</code> option for specifying strings to append to overlapping names in the left and right DataFrame objects:

In [40]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


<table>
    <caption>Table 8.2: <code>pandas.merge</code> function arguments</caption>
    <tr>
        <th>Argument</th>
        <th>Description</th>
    </tr>
    <tr>
        <td><code>left</code></td>
        <td>DataFrame to be merged on the left side.</td>
    </tr>
    <tr>
        <td><code>right</code></td>
        <td>DataFrame to be merged on the right side.</td>
    </tr>
    <tr>
        <td><code>how</code></td>
        <td>Type of join to apply: one of <code>inner</code>, <code>outer</code>, <code>left</code>, or <code>right</code>; defaults to <code>inner</code>.</td>
    </tr>
    <tr>
        <td><code>on</code></td>
        <td>Column names to join on. Must be found in both DataFrame objects. If not specified and no other join keys given, will use the intersection of the column names in <code>left</code> and <code>right</code> as the join keys.</td>
    </tr>
    <tr>
        <td><code>left_on</code></td>
        <td>Columns in left DataFrame to use as join keys. Can be a single column name or a list of column names.</td>
    </tr>
    <tr>
        <td><code>right_on</code></td>
        <td>Analogous to <code<left_on</code> for right DataFrame.</td>
    </tr>
    <tr>
        <td><code>left_index</code></td>
    <td>Use row index in left DataFrame as its join key (or keys, if a <code>MultiIndex</code>).</td>
    </tr>
    <tr>
        <td><code>right_index</code></td>
    <td>Analogous to <code>left_index</code>.</td>
    </tr>
    <tr>
        <td><code>sort</code></td>
    <td>Sort merged data lexicographically by join keys; <code>False</code> by default.</td>
    </tr>
    <tr>
    <td><code><code>suffixes</code></code></td>
        <td>Tuple of string values to append to column names in case of overlap; defaults to <code>("_x", "_y")</code> (e.g., if <code>data</code> in both DataFrame objects, would appear as <code>data_x</code> and <code>data_y</code> in result).</td>
    </tr>
    <tr>
        <td><code>copy</code></td>
    <td>If <code>False</code>, avoid copying data into resulting data structure in some exceptional cases; by default always copies.</td>
    </tr>
    <tr>
        <td><code>validate</code></td>
        <td>Verifies if the merge is of the specified type, whether one-to-one, one-to-many, or many-to-many. See the docstring for full details on the options.</td>
    </tr>
    <tr>
        <td><code>indicator</code></td>
    <td>Adds a special column <code>_merge</code> that indicates the source of each row; values will be <code>left_only</code>, <code>right_only</code>, or <code>both</code> based on the origin of the joined data in each row.</td>
    </tr>
</table>

<h3>Merging on Index</h3>
<p>In some cases, the merge key(s) in a DataFrame will be found in its index (row labels). In this case, you can pass <code>left_index=True</code> or <code>right_index=True</code> (or both) to indicate that the index should be used as the merge key:
    </p>

In [41]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [42]:
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])
right1

,group_val
a,3.5
b,7.0


In [43]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


<p><strong>Note</strong></p>
<p>If you look carefully here, you will see that the index values for <code>left1</code> have been preserved, whereas in other examples above, the indexes of the input DataFrame objects are dropped. Because the index of <code>right1</code> is unique, this "many-to-one" merge (with the default <code>how="inner"</code> method) can preserve the index values from <code>left1</code> that correspond to rows in the output.</p>
<p>Since the default merge method is to intersect the join keys, you can instead form the union of them with an outer join:</p>

In [44]:
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


<p>With hierarchically indexed data, things are more complicated, as joining on index is equivalent to a multiple-key merge:</p>

In [45]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
                               "Nevada", "Nevada"],
                      "key2": [2000, 2001, 2002, 2001, 2002],
                      "data": pd.Series(range(5), dtype="Int64")})
lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [46]:
righth_index = pd.MultiIndex.from_arrays(
    [
        ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)
righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64",
                                           index=righth_index),
                       "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64",
                                           index=righth_index)})
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In this case, you have to indicate multiple columns to merge on as a list (note the handling of duplicate index values with <code>how="outer"</code>):

In [47]:
pd.merge(lefth, righth, left_on=["key1", "key2"], 
         right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [48]:
pd.merge(lefth, righth, left_on=["key1", "key2"],
         right_index=True, how="outer")

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
4,Nevada,2000,<NA>,2,3


<p>Using the indexes of both sides of the merge is also possible:</p>

In [49]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"],
                     columns=["Ohio", "Nevada"]).astype("Int64")
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [50]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=["b", "c", "d", "e"],
                      columns=["Missouri", "Alabama"]).astype("Int64")
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [51]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


<p>DataFrame has a <code>join</code> instance method to simplify merging by index. It can also be used to combine many DataFrame objects having the same or similar indexes but nonoverlapping columns. In the prior example, we could have written:</p>

In [52]:
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


<p>Compared with <code>pandas.merge</code>, DataFrame’s <code>join</code> method performs a left join on the join keys by default. It also supports joining the index of the passed DataFrame on one of the columns of the calling DataFrame:</p>

In [53]:
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


<p>You can think of this method as joining data "into" the object whose <code>join</code> method was called.</p>

<p>Lastly, for simple index-on-index merges, you can pass a list of DataFrames to <code>join</code> as an alternative to using the more general <code>pandas.concat</code> function described in the next section:</p>

In [54]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=["a", "c", "e", "f"],
                       columns=["New York", "Oregon"])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [55]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0


In [56]:
left2.join([right2, another], how="outer")

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0
b,<NA>,<NA>,7,8,NaN,NaN
d,<NA>,<NA>,11,12,NaN,NaN
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0


<h3>Concatenating Along an Axis</h3>
<p>Another kind of data combination operation is referred to interchangeably as <em>concatenation</em> or <em>stacking</em>. NumPy's <code>concatenate</code> function can do this with NumPy arrays:</p>

In [57]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [58]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

<p>In the context of pandas objects such as Series and DataFrame, having labeled axes enable you to further generalize array concatenation. In particular, you have a number of additional concerns:</p>
<ul>
    <li>If the objects are indexed differently on the other axes, should we combine the distinct elements in these axes or use only the values in common?</li>
    <li>Do the concatenated chunks of data need to be identifiable as such in the resulting object?</li>
    <li>Does the "concatenation axis" contain data that needs to be preserved? In many cases, the default integer labels in a DataFrame are best discarded during concatenation.</li>
</ul>
<p>The <code>concat</code> function in pandas provides a consistent way to address each of these questions. I’ll give a number of examples to illustrate how it works. Suppose we have three Series with no index overlap:

In [59]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s1

a    0
b    1
dtype: Int64

In [60]:
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s2

c    2
d    3
e    4
dtype: Int64

In [61]:
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")
s3

f    5
g    6
dtype: Int64

Calling <code>pandas.concat</code> with these objects in a list glues together the values and indexes:

In [62]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

<p>By default, <code>pandas.concat</code> works along <code>axis="index"</code>, producing another Series. If you pass <code>axis="columns"</code>, the result will instead be a DataFrame:</p>

In [63]:
pd.concat([s1, s2, s3], axis="columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


<p>In this case there is no overlap on the other axis, which as you can see is the union (the <code>"outer"</code> join) of the indexes. You can instead intersect them by passing <code>join="inner"</code>:</p>

In [64]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: Int64

In [65]:
pd.concat([s1, s4], axis="columns")

,0,1
a,0,0
b,1,1
f,<NA>,5
g,<NA>,6


In [66]:
pd.concat([s1, s4], axis="columns", join="inner")

,0,1
a,0,0
b,1,1


<p>In this last example, the <code>f</code> and <code>g</code> labels disappeared because of the <code>join="inner"</code> option.</p>

<p>A potential issue is that the concatenated pieces are not identifiable in the result. Suppose instead you wanted to create a hierarchical index on the concatenation axis. To do this, use the <code>keys</code> argument:</p>

In [67]:
result = pd.concat([s1, s1, s3], keys=["one", "two", "three"])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: Int64

In [68]:
result.unstack()

,a,b,f,g
one,0,1,<NA>,<NA>
two,0,1,<NA>,<NA>
three,<NA>,<NA>,5,6


<p>In the case of combining Series along <code>axis="columns"</code>, the <code>keys</code> become the DataFrame column headers:

In [69]:
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


<p>The same logic extends to DataFrame objects:</p>

In [70]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), 
                   index=["a", "b", "c"],
                   columns=["one", "two"])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [71]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), 
                   index=["a", "c"],
                   columns=["three", "four"])
df2

,three,four
a,5,6
c,7,8


In [72]:
pd.concat([df1, df2], axis="columns", 
          keys=["level1", "level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

<p>Here the <code>keys</code> argument is used to create a hierarchical index where the first level can be used to identify each of the concatenated DataFrame objects.</p>

<p>If you pass a dictionary of objects instead of a list, the dictionary’s keys will be used for the <code>keys</code> option:</p>

In [73]:
pd.concat({"level1": df1, "level2": df2}, axis="columns")

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

<p>There are additional arguments governing how the hierarchical index is created (see <a href="#Table_8.3">Table 8.3</a>). For example, we can name the created axis levels with the <code>names</code> argument:</p>

In [74]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"],
          names=["upper", "lower"])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

<p>A last consideration concerns DataFrames in which the row index does not contain any relevant data:</p>

In [75]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)),
                   columns=["a", "b", "c", "d"])
df1

,a,b,c,d
0,1.248804,0.774191,-0.319657,-0.624964
1,1.078814,0.544647,0.855588,1.343268
2,-0.267175,1.793095,-0.652929,-1.886837


In [76]:
df2 = pd.DataFrame(np.random.standard_normal((2, 3)),
                   columns=["b", "d", "a"])
df2

,b,d,a
0,1.059626,0.644448,-0.007799
1,-0.449204,2.448963,0.667226


<p>In this case, you can pass <code>ignore_index=True</code>, which discards the indexes from each DataFrame and concatenates the data in the columns only, assigning a new default index:</p>

In [77]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,1.248804,0.774191,-0.319657,-0.624964
1,1.078814,0.544647,0.855588,1.343268
2,-0.267175,1.793095,-0.652929,-1.886837
3,-0.007799,1.059626,NaN,0.644448
4,0.667226,-0.449204,NaN,2.448963


<table id='Table_8.3'>
    <caption>Table 8.3: <code>pandas.concat</code> function arguments</caption>
    <tr>
        <th>Argument</th>
        <th>Description</th>
    </tr>
    <tr>
        <td><code>objs</code></td>
        <td>List or dictionary of pandas objects to be concatenated; this is the only required argument</td>
    </tr>
    <tr>
        <td><code>axis</code></td>
        <td>Axis to concatenate along; defaults to concatenating along rows (<code>axis="index"</code>)</td>
    </tr>
    <tr>
        <td><code>join</code></td>
        <td>Either <code>inner</code> or <code>outer</code> (<code>outer</code> by default); whether to intersect (inner) or union (outer) indexes along the other axes</td>
    </tr>
    <tr>
        <td><code>keys</code></td>
        <td>Values to associate with objects being concatenated, forming a hierarchical index along the concatenation axis; can be a list or array of arbitrary values, an array of tuples, or a list of arrays (if multiple-level arrays passed in <code>levels</code>)</td>
    </tr>
    <tr>
        <td><code>levels</code></td>
        <td>Specific indexes to use as hierarchical index level or levels if keys passed</td>
    </tr>
    <tr>
        <td><code>names</code></td>
        <td>Names for created hierarchical levels if <code>keys</code> and/or <code>levels</code> passed</td>
    </tr>
    <tr>
        <td><code>verify_integrity</code></td>
        <td>Check new axis in concatenated object for duplicates and raise an exception if so; by default (<code>False</code>) allows duplicates</td>
    </tr>
    <tr>
        <td><code>ignore_index</code></td>
        <td>Do not preserve indexes along concatenation <code>axis</code>, instead produce a new <code>range(total_length)</code> index</td>
    </tr>
</table>
<h3>Combining Data with Overlap</h3>
<p>There is another data combination situation that can’t be expressed as either a merge or concatenation operation. You may have two datasets with indexes that overlap in full or in part. As a motivating example, consider NumPy’s <code>where</code> function, which performs the array-oriented equivalent of an if-else expression:</p>

In [78]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
              index=["f", "e", "d", "c", "b", "a"])
a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [79]:
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
              index=["a", "b", "c", "d", "e", "f"])
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [80]:
np.where(pd.isna(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

<p>Here, whenever values in <code>a</code> are null, values from <code>b</code> are selected, otherwise the non-null values from <code>a</code> are selected. Using <code>numpy.where</code> does not check whether the index labels are aligned or not (and does not even require the objects to be the same length), so if you want to line up values by index, use the Series <code>combine_first</code> method:</p>

In [81]:
a.combine_first(b)

a    0.0
b    4.5
c    3.5
d    0.0
e    2.5
f    5.0
dtype: float64

<p>With DataFrames, <code>combine_first</code> does the same thing column by column, so you can think of it as “patching” missing data in the calling object with data from the object you pass:</p>

In [82]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
                    "b": [np.nan, 2., np.nan, 6.],
                    "c": range(2, 18, 4)})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [83]:
df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                    "b": [np.nan, 3., 4., 6., 8.]})
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [84]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


<p>The output of <code>combine_first</code> with DataFrame objects will have the union of all the column names.</p>

<h2>8.3 Reshaping and Pivoting</h2>

<p>There are a number of basic operations for rearranging tabular data. These are referred to as <em>reshape</em> or <em>pivot</em> operations.</p>
<h3>Reshaping with Hierarchical Indexing</h3>

<p>Hierarchical indexing provides a consistent way to rearrange data in a DataFrame. There are two primary actions:</p>

<dl>
    <dt>stack</dt>
    <dd>This “rotates” or pivots from the columns in the data to the rows.</dd>
    <dt>unstack</dt>
    <dd>This pivots from the rows into the columns.</dd>
</dl>

<p>I’ll illustrate these operations through a series of examples. Consider a small DataFrame with string arrays as row and column indexes:</p>

In [85]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"],
                    name="number"))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


<p>Using the <code>stack</code> method on this data pivots the columns into the rows, producing a Series:</p>

In [86]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

<p>From a hierarchically indexed Series, you can rearrange the data back into a DataFrame with <code>unstack</code>:</p>

In [87]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


<p>By default, the innermost level is unstacked (same with <code>stack</code>). You can unstack a different level by passing a level number or name:</p>

In [88]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [89]:
result.unstack(level="state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


<p>Unstacking might introduce missing data if all of the values in the level aren’t found in each subgroup:</p>

In [90]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")
data2 = pd.concat([s1, s2], keys=["one", "two"])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

<p>Stacking filters out missing data by default, so the operation is more easily invertible:</p>

In [91]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


In [92]:
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [93]:
data2.unstack().stack(dropna=False)

one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

<p>When you unstack in a DataFrame, the level unstacked becomes the lowest level in the result:</p>

In [94]:
df = pd.DataFrame({"left": result, "right": result + 5},
                  columns=pd.Index(["left", "right"], name="side"))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [95]:
df.unstack(level="state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

<p>As with <code>unstack</code>, when calling <code>stack</code> we can indicate the name of the axis to stack:</p>

In [96]:
df.unstack(level="state").stack(level="side")

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

<h3>Pivoting “Long” to “Wide” Format</h3>

<p>A common way to store multiple time series in databases and CSV files is what is sometimes called long or stacked format. In this format, individual values are represented by a single row in a table rather than multiple values per row.</p>

<p>Let's load some example data and do a small amount of time series wrangling and other data cleaning:</p>

In [97]:
data = pd.read_csv("examples/macrodata.csv")
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]
data.head()

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2


<p>First, I use <code>pandas.PeriodIndex</code> (which represents time intervals rather than points in time), discussed in more detail in Ch 11: Time Series, to combine the <code>year</code> and <code>quarter</code> columns to set the index to consist of <code>datetime</code> values at the end of each quarter:</p>

In [98]:
periods = pd.PeriodIndex(year=data.pop("year"),
                         quarter=data.pop("quarter"),
                         name="date")
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [99]:
data.index = periods.to_timestamp("D")
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


<p>Here I used the <code>pop</code> method on the DataFrame, which returns a column while deleting it from the DataFrame at the same time.</p>

<p>Then, I select a subset of columns and give the columns index the name "item":</p>

In [100]:
data = data.reindex(columns=["realgdp", "infl", "unemp"])
data.columns.name = "item"
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


<p>Lastly, I reshape with <code>stack</code>, turn the new index levels into columns with <code>reset_index</code>, and finally give the column containing the data values the name <code>value</code>:</p>

In [101]:
long_data = (data.stack()
             .reset_index()
             .rename(columns={0: "value"}))

<p>Now, <code>ldata</code> looks like:</p>

In [102]:
long_data[:10]

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


<p>In this so-called long format for multiple time series, each row in the table represents a single observation.</p>

<p>Data is frequently stored this way in relational SQL databases, as a fixed schema (column names and data types) allows the number of distinct values in the <code>item</code> column to change as data is added to the table. In the previous example, <code>date</code> and <code>item</code> would usually be the primary keys (in relational database parlance), offering both relational integrity and easier joins. In some cases, the data may be more difficult to work with in this format; you might prefer to have a DataFrame containing one column per distinct <code>item</code> value indexed by timestamps in the date column. DataFrame’s <code>pivot</code> method performs exactly this transformation:</p>

In [103]:
pivoted = long_data.pivot(index="date", columns="item",
                          values="value")
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


<p>The first two values passed are the columns to be used, respectively, as the row and column index, then finally an optional value column to fill the DataFrame. Suppose you had two value columns that you wanted to reshape simultaneously:</p>

In [104]:
long_data["value2"] = np.random.standard_normal(len(long_data))
long_data[:10]

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,0.802926
1,1959-01-01,infl,0.000,0.575721
2,1959-01-01,unemp,5.800,1.381918
3,1959-04-01,realgdp,2778.801,0.000992
4,1959-04-01,infl,2.340,-0.143492
5,1959-04-01,unemp,5.100,-0.206282
6,1959-07-01,realgdp,2775.488,-0.222392
7,1959-07-01,infl,2.740,-1.682403
8,1959-07-01,unemp,5.300,1.811659
9,1959-10-01,realgdp,2785.204,-0.351305


<p>By omitting the last argument, you obtain a DataFrame with hierarchical columns:</p>

In [105]:
pivoted = long_data.pivot(index="date", columns="item")
pivoted.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.575721  0.802926  1.381918
1959-04-01  2.34  2778.801   5.1 -0.143492  0.000992 -0.206282
1959-07-01  2.74  2775.488   5.3 -1.682403 -0.222392  1.811659
1959-10-01  0.27  2785.204   5.6  0.128317 -0.351305 -1.313554
1960-01-01  2.31  2847.699   5.2 -0.615939  0.498327  0.174072

In [106]:
pivoted["value"].head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


<p>Note that <code>pivot</code> is equivalent to creating a hierarchical index using <code>set_index</code> followed by a call to <code>unstack</code>:</p>

In [107]:
unstacked = long_data.set_index(["date", "item"]).unstack(level="item")
unstacked.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.575721  0.802926  1.381918
1959-04-01  2.34  2778.801   5.1 -0.143492  0.000992 -0.206282
1959-07-01  2.74  2775.488   5.3 -1.682403 -0.222392  1.811659
1959-10-01  0.27  2785.204   5.6  0.128317 -0.351305 -1.313554
1960-01-01  2.31  2847.699   5.2 -0.615939  0.498327  0.174072

<h3>Pivoting “Wide” to “Long” Format</h3>

<p>An inverse operation to <code>pivot</code> for DataFrames is <code>pandas.melt</code>. Rather than transforming one column into many in a new DataFrame, it merges multiple columns into one, producing a DataFrame that is longer than the input. Let's look at an example:</p>

In [108]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                   "A": [1, 2, 3],
                   "B": [4, 5, 6],
                   "C": [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


<p>The <code>key</code> column may be a group indicator, and the other columns are data values. When using <code>pandas.melt</code>, we must indicate which columns (if any) are group indicators. Let's use <code>key</code> as the only group indicator here:</p>

In [109]:
melted = pd.melt(df, id_vars="key")
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


<p>Using <code>pivot</code>, we can reshape back to the original layout:</p>

In [110]:
reshaped = melted.pivot(index="key", columns="variable",
                        values="value")
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


<p>Since the result of <code>pivot</code> creates an index from the column used as the row labels, we may want to use <code>reset_index</code> to move the data back into a column:</p>

In [111]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


<p>You can also specify a subset of columns to use as <code>value</code> columns:</p>

In [112]:
pd.melt(df, id_vars="key", value_vars=["A", "B"])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


<p><code>pandas.melt</code> can be used without any group identifiers, too:</p>

In [113]:
pd.melt(df, value_vars=["A", "B", "C"])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [114]:
pd.melt(df, value_vars=["key", "A", "B"])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
